In [1]:
## Importing libraries

import pandas as pd
import numpy as np
import sys; sys.path.insert(1, '/Users/EvanCallaghan/Documents/Golf/StrokesGained/GOLF/GlobalScripts')
pd.set_option('display.max_rows', None, 'display.max_columns', None)
import sg_script as sg
import sg_script_editor as sg_editor
import sg_script_multiple_rounds as sg_multiple

In [2]:
## Reading the data

baselines = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Baselines.xlsx')
courses = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Courses.xlsx')
golfers = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Golfers.xlsx')
rounds = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Rounds.xlsx')
shots = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Shots.xlsx')

In [4]:
## Subsetting the data for the correct golfer and round
golfer = golfers[golfers['Name'] == 'John David Coffin'].reset_index(drop = True)
rd = rounds[rounds['roundID'] == 1056].reset_index(drop = True)
shot = shots[shots['roundID'] == 1056].reset_index(drop = True)
course = courses[(courses['courseID'] == rd.at[0, 'courseID']) & (courses['Tees'] == rd.at[0, 'Tees'])].reset_index(drop = True)

## Creating complete data-frame for analysis
shotLevel, roundLevel = sg.initialize(shot, rd, course, baselines)

## Exporting data-frames for analysis:
shotLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/shotLevel.csv', index = False)
roundLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/roundLevel.csv', index = False)
course.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/courseLevel.csv', index = False)

### Multiple round analysis

In [3]:
## Subsetting the data for the correct golfer and rounds
golfer = golfers[golfers['Name'] == 'John David Coffin'].reset_index(drop = True)
rds = rounds[(rounds['Name'] == 'John David Coffin') & (rounds['roundID'] >= 1010)].reset_index(drop = True)
shot = shots[np.isin(shots['roundID'], rds['roundID'].unique())].reset_index(drop = True)

rds['courseTeeCombo'] = rds['Course'] + rds['Tees']
courses['courseTeeCombo'] = courses['Course'] + courses['Tees']
course = courses[np.isin(courses['courseTeeCombo'], rds['courseTeeCombo'].unique())].reset_index(drop = True)

## Creating complete data-frame for analysis
shotLevel, roundLevel = sg_multiple.initialize(shot, rds, course, baselines)

# ## Exporting data-frames for analysis:
# shotLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/shotLevelMultiple.csv', index = False)
# roundLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/roundLevelMultiple.csv', index = False)

In [5]:
roundLevel

,roundID,golferID,Name,Date,courseID,Course,Tees,holesPlayed,puttDistance,roundLength,Weather,tournamentRound,Tournament,Notes,courseTeeCombo,APP,ATG,PUTT,OTT,Score,SG: Total,Fairway,GIR,Putts,Par,scoreToPar
0,1024,7,John David Coffin,2022-05-28,8,Ken-Wo,Alternate,18,Yards,4.5,Rain/Wind/Sun,1,Ken-Wo Apple Blossom,NaN,Ken-WoAlternate,-4.08,0.07,0.85,-0.47,72,-3.63,11,9,28,69,3
1,1025,7,John David Coffin,2022-05-29,8,Ken-Wo,Alternate,18,Yards,4.5,Sun,2,Ken-Wo Apple Blossom,NaN,Ken-WoAlternate,0.03,-1.65,-4.76,-2.24,76,-8.62,9,12,34,69,7
2,1026,7,John David Coffin,2022-06-01,9,Abercrombie,Black,18,Yards,4.5,Cold,0,NaN,Men's league,AbercrombieBlack,-3.70,-1.03,-3.24,-2.09,79,-10.06,7,11,34,71,8
3,1054,7,John David Coffin,2022-06-30,14,Dundarave,Black,18,Yards,6.0,Sun,1,PGA Tour Canada: PEI Open,Round 1,DundaraveBlack,-2.07,-4.09,-1.53,-8.18,87,-15.87,7,9,33,72,15
4,1055,7,John David Coffin,2022-07-01,14,Dundarave,Black,18,Yards,6.0,Sun,1,PGA Tour Canada: PEI Open,Round 2,DundaraveBlack,-1.71,-1.77,-3.71,-1.74,80,-8.93,6,8,32,72,8
5,1056,7,John David Coffin,2022-07-17,9,Abercrombie,Blue,18,Yards,4.0,Sun,0,NaN,NaN,AbercrombieBlue,-1.49,0.64,-3.10,0.93,71,-3.02,7,12,30,71,0
6,1059,7,John David Coffin,2022-07-20,9,Abercrombie,Blue,18,Feet,4.0,Sun,0,NaN,NaN,AbercrombieBlue,0.09,0.08,-2.09,-0.24,71,-2.16,8,13,31,71,0


In [9]:
pd.DataFrame(roundLevel[['SG: Total', 'OTT', 'APP', 'ATG', 'PUTT']].mean())

,0
SG: Total,-7.470000
OTT,-2.004286
APP,-1.847143
ATG,-1.107143
PUTT,-2.511429


In [12]:
pd.DataFrame(roundLevel[roundLevel['Notes'] != 'Round 1'][['SG: Total', 'OTT', 'APP', 'ATG', 'PUTT']].mean())

,0
SG: Total,-6.070
OTT,-0.975
APP,-1.810
ATG,-0.610
PUTT,-2.675
